# Import Modules

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from scipy.optimize import differential_evolution
from scipy.optimize import basinhopping

# Functions

## basic generators

In [ ]:
def allgen(alpha, Nb = 10000, model = '0'):
    X_test = np.zeros([Nb,24])
    X_forc = np.zeros([Nb,24])
    y_test = np.zeros([Nb,1]) 
    steps = 25
    count = 0
    probability = .95
    for tt in range(0,50):
        Wx_pred6 = pred[tt,:,:,:]
        Wx_real6 = real[tt,:,:,:]
        for i in range(80,100):
            for j in range(80,100):
                lat = i
                long = j
                #X_test[count,:] = newgen(alpha, probability, long,lat,Wx_pred6,Wx_real6)
                X_test[count,:],X_forc[count,:] = newgen_mo(alpha, probability, long,lat,Wx_pred6,Wx_real6)
                #y_test[count,:] = 0
                count += 1
                #if count%1000 == 0:
                    #print(count,end = ',')
                if count  >= Nb:
                    X_test = X_test[:count,:]
                    X_tforc = X_forc[:count,:]
                    #y_test = y_test[:count,:]
                    #print('\n Finished')
                    return X_test,X_forc
    #print('\n Finished')
    return X_test

def newgen(alpha, prob,long,lat,Wx_pred6,Wx_real6):
    cte = 3
    steps = 25
    y_false = []
    i = lat
    j = long
    ecart_1 = 0
    for t in range(1,steps):
        oneforc = Wx_pred6[i,j,t]
        yreal = Wx_real6[i,j,t]
        oneforc = oneforc* (1+ alpha[0]*ecart_1)
        sigma_1 = sigma[i,j]*hour[t]*cte
        
        if t > 3:
            sigma_1 = sigma[i,j]*hour[3]*cte*alpha[1]
        
        #sigma_1 = sigma[i,j]*(np.e**-3.2*hour[t])*cte
        val = norm.rvs(oneforc,sigma_1)
        #va,vb = norm.interval(prob,oneforc,sigma_1)
        #while val < va or val > vb:
        #    val = norm.rvs(oneforc,sigma_1)
        y_false += [val]
        ecart_1 = (val-oneforc)/np.abs(oneforc)
        if oneforc == 0:
            ecart_1 = 0
        i = i+1
        
    return y_false

def newgen_mo(alpha, prob,long,lat,Wx_pred6,Wx_real6):
    cte = 3
    steps = 25
    y_false = []
    forcs = []
    i = lat
    j = long
    ecart_1 = 0
    for t in range(1,steps):
        oneforc = Wx_pred6[i,j,t]
        forcs += [oneforc]
        #yreal = Wx_real6[i,j,t]
        oneforc = oneforc* (1+ alpha[0]*ecart_1)
        sigma_1 = sigma[i,j]*hour[t]*cte
        
        if t > 3:
            sigma_1 = sigma[i,j]*hour[3]*cte*alpha[1]
        
        #sigma_1 = sigma[i,j]*(np.e**-3.2*hour[t])*cte
        val = norm.rvs(oneforc,sigma_1)
        #va,vb = norm.interval(prob,oneforc,sigma_1)
        #while val < va or val > vb:
        #    val = norm.rvs(oneforc,sigma_1)
        y_false += [val]
        ecart_1 = (val-oneforc)/np.abs(oneforc)
        if oneforc == 0:
            ecart_1 = 0
        i = i+1
        
    return y_false,forcs

def predmod(alpha,write = 0):
    X_test,y_forc = allgen(alpha, 10000)
    aa = np.where(clf2.predict(X_test) == 1)[0]
    X_new = X_test[aa,:]
    dist = 0
    for i in range(len(X_new)):
        courbe = X_new[i,:]
        forc = y_forc[i,:]
        for j in range(len(courbe)):
            dist += np.abs(courbe[j]-forc[j])
    #y_test = 1 - clf2.predict(X_test)
    #red = np.sum(y_test)
    if aa.size == 0 :
        rew = 0
        raw = 0
    else :
        rew = dist/(np.sqrt(len(aa)))
        raw = dist/((len(aa)))
    writ = str(rew) + ';' + str(aa.size) + ';' + str(raw) + '|' + str(alpha[0]) + ',' + str(alpha[1]) + '\n'
    if write == 1:
        f.write(writ)
    print(writ)
    return -rew

## handmade generators

In [ ]:
def flagpolicy(flag,val,oneforc,onef):
    '''Tries to input a conditional meaning in the generator'''
    if flag > 0:
        if val > onef:
            flag += 1
        elif val < oneforc:
            flag += -1
    elif flag < 0:
        if val > oneforc:
            flag += 1
        elif val < onef:
            flag += -1
    else:
        if val > oneforc:
            flag += 1
        elif val < oneforc:
            flag += -1
    return flag

def genf(prob, long,lat,Wx_pred6,Wx_real6):
    '''Generator for conditional data based in independent probabilities'''

    cte = 3
    y_false = []
    y_real = []
    forcs = []
    va95 = []
    vb95 = []
    sigma_t = []
    steps = 25

    i = lat 
    j = long
    flag = 0
    
    oneforc0 = Wx_pred6[i,j,0]
    val0 = oneforc0

    for t in range(1,steps):
        oneforc = Wx_pred6[i,j,t]
        yreal = Wx_real6[i,j,t]
        sigma_1 = sigma[i,j]*hour[t]*cte
        va,vb = norm.interval(.5,oneforc,sigma_1)
        val = 1000

        ad = sigma_1*flag/5
        #print(ad)
        va = va + ad
        vb = vb + ad
        onef = oneforc + ad
        flagg = 1
        dforc = oneforc - oneforc0
        while flagg:
            val = norm.rvs(onef,sigma_1)
            if val < va or val > vb:
                dval = val - val0
                if dval*dforc > 0:
                    flagg = 0
        flag = flagpolicy(flag,val,oneforc,onef)

        y_false += [val] 
        y_real += [yreal]
        forcs += [oneforc]
        sigma_t += [sigma_1]
        va,vb = norm.interval(.95,oneforc,sigma_1)
        va95 += [va]
        vb95 += [vb]
        i = i+1
        oneforc0 = oneforc
        val0 = val
    #print(flag)
    return(y_false,y_real,forcs,va95,vb95)

def gen_simp(long,lat,Wx_pred6,Wx_real6):
    cte = 3
    steps = 25
    y_false = []
    i = lat
    j = long
    for t in range(1,steps):
        oneforc = Wx_pred6[i,j,t]
        #print(oneforc)
        sigma_1 = sigma[i,j]*hour[t]*cte
        
        if t > 3:
            sigma_1 = sigma[i,j]*(np.e**-3.2*hour[3])*cte
        
        val = norm.rvs(oneforc,sigma_1)
        y_false += [val]
        i = i+1
    return y_false

def gen_mod(prob,long,lat,Wx_pred6,Wx_real6):
    cte = 3
    steps = 25
    y_false = []
    i = lat
    j = long
    for t in range(1,steps):
        oneforc = Wx_pred6[i,j,t]
        sigma_1 = sigma[i,j]*hour[t]*cte
        val = norm.rvs(oneforc,sigma_1)
        va,vb = norm.interval(prob,oneforc,sigma_1)
        while val < va or val > vb:
            val = norm.rvs(oneforc,sigma_1)
        y_false += [val]
        i = i+1
    return y_false

# Parameters and data

In [ ]:
# Parameters
cte = 0.1223 #constant for sigma
param = 'Wx' #parameter to be evaluated Wx, Wy or T

In [ ]:
#load or save the file with train and test data
file_save = 'Train_test_Data' + param
directory = 'Images/'
filename = directory + file_save + '.npz'
#np.savez(filename, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test)
data = np.load(filename)
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test'] 
y_test = data['y_test'] 
print(filename)
data.close()

#load or save the file with the values of sigma for Wx for each latitude and longitude
file_save = 'sigma_' + param
directory = 'data_matrix/'
filename = directory + file_save + '.npz'
#np.savez(filename, sigma = sigma)
#print(filename)
data = np.load(filename)
sigma = data['sigma']
data.close()

#load or save the file with the observed data
file_save = 'Wind_UVT-level_20000-1Forc_000'
directory = 'data_matrix/'
filename = directory + file_save + '.npz'
#np.savez(filename, Wx_alpha = Wx_15min)
#print(filename)
data = np.load(filename)
Wx_0 = data[param]
time_0 = data['vec_time']
hour_0 = data['delta_hours']
data.close()

#load or save the file with the forecast (+6 hours) data
file_save = 'Wind_UVT-level_20000-1Forc_006'
directory = 'data_matrix/'
filename = directory + file_save + '.npz'
#np.savez(filename, Wx_alpha = Wx_15min)
#print(filename)
data = np.load(filename)
Wx_1 = data[param]
time_1 = data['vec_time']
hour_1 = data['delta_hours']
data.close()

# Pre-processing data

In [ ]:
#Chooses the data so they will be in the same time_step

time0 = []
ii = []
jj = []
for i in range(len(time_0)):
    for j in range(len(time_1)):
        if time_0[i] == time_1[j]:
            time0 += [time_0]
            ii += [i]
            jj += [j]

tim_0 = time_0[ii]
tim_1 = time_1[jj]
Wx_0 = Wx_0[:,:,ii]
Wx_1 = Wx_1[:,:,jj]
delta_hours = (tim_0[1:] - tim_1[:-1])
n = len(delta_hours)
dhour = np.zeros([n,1])
for i in range(0,n):
    dhour[i] = delta_hours[i].total_seconds()/3600
    
sh_i = np.shape(Wx_0)[0]
sh_j = np.shape(Wx_0)[1]
sh_tt = np.shape(Wx_0)[2]
sh_t = 25+1
Wx_pred6 = np.zeros([sh_i,sh_j,sh_t])
Wx_real6 = np.zeros([sh_i,sh_j,sh_t])
hour = []
i = 1
pred = []
real = []
for i in range(sh_tt-1):
    for j in range(sh_t-1):
        #Do a linear interpolation in the data to "forcast" to anytime 
        #between 0 and 6, to be chosen in the variable hours [h]
        hours = .25*j
        tt = hours/6
        Wx_pred6[:,:,j] = (Wx_1[:,:,i+1]-Wx_0[:,:,i])*tt+Wx_0[:,:,i]
        Wx_real6[:,:,j] = (Wx_0[:,:,i+1]-Wx_0[:,:,i])*tt+Wx_0[:,:,i]    
        hour += [hours]
    pred +=  [Wx_pred6[:,:,:-1]]
    real +=  [Wx_real6[:,:,:-1]]

pred = np.array(pred)
real = np.array(real)   

# Classifiers

In [ ]:
#Random Forest Train
clf2 = RandomForestClassifier(n_estimators=20,verbose = 0)#,min_impurity_decrease=.0001)#,max_depth = 15)
clf2.fit(X_train, np.ravel(y_train))
y_pred = clf2.predict(X_train)
[tn, fp], [fn, tp] = confusion_matrix(y_train, y_pred)
print(tn,fp,'\n',fn,tp)
print('True positives:',tp,'True negatives:',tn,'False positives:',fp, 'False negatives:',fn)

# Optimizing the generator

In [ ]:
#Differential evolution
#f = open("Gado_dif_evo_2alpha.txt","w+")
bounds = [(-5, 5),(.5,5)]
result = differential_evolution(predmod, bounds, polish = True, disp = True)
#f.close() 
print('OVER!!!')
print(result)
alpha = result.x

# Creating the new training/test data set

In [ ]:
#test dataset
#alpha = result.x
Nb = 60000
X_test = np.zeros([Nb,24])
y_test = np.zeros([Nb,1]) 
steps = 25
count = 0
probability = .95
#for t in range(0,20):
for tt in range(50,len(pred)):
    Wx_pred6 = pred[tt,:,:,:]
    Wx_real6 = real[tt,:,:,:]
    for i in range(80,100):
        for j in range(80,100):
            lat = i
            long = j
            y_false,y_real,forcs,va95,vb95 = genf(.95,long,lat,Wx_pred6,Wx_real6)
            X_test[count,:] = y_real
            y_test[count,:] = 1
            count += 1
            X_test[count,:] = newgen(alpha, probability, long,lat,Wx_pred6,Wx_real6)
            #X_test[count,:] = y_false
            y_test[count,:] = 0
            count += 1
            X_test[count,:] = forcs
            y_test[count,:] = 1
            count += 1
            X_test[count,:] = newgen(alpha, probability, long,lat,Wx_pred6,Wx_real6)
            #X_test[count,:] = gen_mod(probability, long,lat,Wx_pred6,Wx_real6)
            y_test[count,:] = 0
            count += 1
            if count%1000 == 0:
                print(count,end = ',')
            if count+5 > Nb:
                print('oi')
                break

X_test = X_test[:count,:]
y_test = y_test[:count,:]
print('\n Finished')

In [ ]:
y_pred = clf2.predict(X_test)
[tn, fp], [fn, tp] = confusion_matrix(y_test, y_pred)
print(tn,fp,'\n',fn,tp)
print('True positives:',tp,'True negatives:',tn,'False positives:',fp, 'False negatives:',fn)